In [1]:
#Libraries
import requests
import json
import time
import datetime as dt
import numpy as np
import pandas as pd
import seaborn as sea


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\Rodolfo\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## 0. Requesting Flypside Data by ShroomDK

In [4]:
def ShroomDK(api_key,query):
    
    API_KEY =api_key
    SQL_QUERY = query

    TTL_MINUTES = 15

    # return up to 100,000 results per GET request on the query id
    PAGE_SIZE = 100000

    # return results of page 1
    PAGE_NUMBER = 1

    def create_query():
        r = requests.post(
            'https://node-api.flipsidecrypto.com/queries', 
            data=json.dumps({
                "sql": SQL_QUERY,
                "ttlMinutes": TTL_MINUTES
            }),
            headers={"Accept": "application/json", "Content-Type": "application/json", "x-api-key":API_KEY},
        )
        if r.status_code != 200:
            raise Exception("Error creating query, got response: " + r.text + "with status code: " + str(r.status_code))

        return json.loads(r.text)    


    def get_query_results(token):
        r = requests.get(
            'https://node-api.flipsidecrypto.com/queries/{token}?pageNumber={page_number}&pageSize={page_size}'.format(
              token=token,
              page_number=PAGE_NUMBER,
              page_size=PAGE_SIZE
            ),
            headers={"Accept": "application/json", "Content-Type": "application/json", "x-api-key": API_KEY}
        )
        if r.status_code != 200:
            raise Exception("Error getting query results, got response: " + r.text + "with status code: " + str(r.status_code))

        data = json.loads(r.text)
        if data['status'] == 'running':
            time.sleep(10)
            return get_query_results(token)

        return data


    def run():
        query = create_query()
        token = query.get('token')
        data = get_query_results(token)

        #print(data['columnLabels'])
        list_data = []
        for row in data['results']:
            #print(row)
            list_data.append(row)
        df = pd.DataFrame(data =list_data, columns = data['columnLabels'])
        return df
    
    return run()

### 1. Time Between Blocks - Last 30 Days

In [5]:
api_key = '33552129-3743-4219-8d00-ebe02f43878b'
query = """
---------------------------------------Polygon--------------------------------------------------------------------------
WITH POLYGON_PERFORMANCE AS (
  WITH BLOCKCHAIN_TX AS (
    SELECT
    	BLOCK_ID,
    	COUNT(DISTINCT(TX_ID)) AS TX_AMOUNT,
    	MIN(BLOCK_TIMESTAMP) AS BLOCK_TIME
    	--SUM(FEE_USD) AS TOTAL_FEE_USD
    FROM
     flipside_prod_db.polygon.transactions
    WHERE DATE_TRUNC('day',BLOCK_TIMESTAMP) >= CURRENT_DATE - INTERVAL '30days'
    	--AND SUCCESS = TRUE --EXPLAIN IT IN CONSTRAINTS AND FILTERS SECTION
    GROUP BY 1
    ORDER BY 2 ASC),
  
  BLOCK_TIME_TAB AS (
  SELECT
  	*,
    	LAG(BLOCK_TIME) OVER(ORDER BY BLOCK_TIME) AS PREVIOUS_BLOCK_TIME,
    	COALESCE(DATEDIFF('second',PREVIOUS_BLOCK_TIME,BLOCK_TIME),0) AS BLOCK_TIME_DIFF
  FROM BLOCKCHAIN_TX 
  ORDER BY 1)
  
  SELECT
  'L2' AS LAYER,
  'POLYGON' AS BLOCKCHAIN,
  AVG(TX_AMOUNT) AS AVG_TX_AMOUNT,
  MAX(TX_AMOUNT) AS MAX_TX_AMOUNT,
  MIN(TX_AMOUNT) AS MIN_TX_AMOUNT,
  AVG(BLOCK_TIME_DIFF) AS AVG_TIME_DIFF,
  PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY BLOCK_TIME_DIFF) AS MEDIAN_TIME_DIFF,
  MAX(BLOCK_TIME_DIFF) AS MAX_TIME_DIFF,
  MIN(BLOCK_TIME_DIFF) AS MIN_TIME_DIFF,
  AVG_TX_AMOUNT/AVG_TIME_DIFF AS AVG_SPEED,
  AVG_TX_AMOUNT/MEDIAN_TIME_DIFF AS MEDIAN_SPEED
  FROM
  	BLOCK_TIME_TAB),

---------------------------------------SOLANA--------------------------------------------------------------------------
SOLANA_PERFORMANCE AS (
  WITH BLOCKCHAIN_TX AS (
    SELECT
    	BLOCK_ID,
    	COUNT(DISTINCT(TX_ID)) AS TX_AMOUNT,
    	MIN(BLOCK_TIMESTAMP) AS BLOCK_TIME
    	--SUM(FEE_USD) AS TOTAL_FEE_USD
    FROM
     solana.core.fact_transactions
    WHERE DATE_TRUNC('day',BLOCK_TIMESTAMP) >= CURRENT_DATE - 30
    	--AND SUCCEEDED = TRUE --EXPLAIN IT IN CONSTRAINTS AND FILTERS SECTION
    GROUP BY 1
    ORDER BY 2 ASC),
  
  BLOCK_TIME_TAB AS (
  SELECT
  	*,
    LAG(BLOCK_TIME) OVER(ORDER BY BLOCK_TIME) AS PREVIOUS_BLOCK_TIME,
    COALESCE(DATEDIFF('second',PREVIOUS_BLOCK_TIME,BLOCK_TIME),0) AS BLOCK_TIME_DIFF
  FROM BLOCKCHAIN_TX
  ORDER BY 1)
  
  SELECT
  	'L1' AS LAYER,
    'SOLANA' AS BLOCKCHAIN,
    AVG(TX_AMOUNT) AS AVG_TX_AMOUNT,
    MAX(TX_AMOUNT) AS MAX_TX_AMOUNT,
    MIN(TX_AMOUNT) AS MIN_TX_AMOUNT,
    AVG(BLOCK_TIME_DIFF) AS AVG_TIME_DIFF,
  	PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY BLOCK_TIME_DIFF) AS MEDIAN_TIME_DIFF,
    MAX(BLOCK_TIME_DIFF) AS MAX_TIME_DIFF,
    MIN(BLOCK_TIME_DIFF) AS MIN_TIME_DIFF,
    AVG_TX_AMOUNT/AVG_TIME_DIFF AS AVG_SPEED,
  	AVG_TX_AMOUNT/MEDIAN_TIME_DIFF AS MEDIAN_SPEED
  FROM
  	BLOCK_TIME_TAB),

-------------------------------------------------FLOW------------------------------------------------------------
FLOW_PERFORMANCE AS (
  WITH BLOCKCHAIN_TX AS (
  SELECT
  	BLOCK_HEIGHT,
  	COUNT(DISTINCT(TX_ID)) AS TX_AMOUNT,
  	MIN(BLOCK_TIMESTAMP) AS BLOCK_TIME
  FROM
   flow.core.fact_transactions
  WHERE DATE_TRUNC('day',BLOCK_TIMESTAMP) >= CURRENT_DATE - 30
  	--AND TX_SUCCEEDED = TRUE --EXPLAIN IT IN CONSTRAINTS AND FILTERS SECTION
  GROUP BY 1
  ORDER BY 2 ASC),
  
  BLOCK_TIME_TAB AS (
  SELECT
  	*,
    LAG(BLOCK_TIME) OVER(ORDER BY BLOCK_TIME) AS PREVIOUS_BLOCK_TIME,
    COALESCE(DATEDIFF('second',PREVIOUS_BLOCK_TIME,BLOCK_TIME),0) AS BLOCK_TIME_DIFF
  FROM BLOCKCHAIN_TX
  ORDER BY 1)
  
  SELECT
    'L1' AS LAYER,
    'FLOW' AS BLOCKCHAIN,
    AVG(TX_AMOUNT) AS AVG_TX_AMOUNT,
    MAX(TX_AMOUNT) AS MAX_TX_AMOUNT,
    MIN(TX_AMOUNT) AS MIN_TX_AMOUNT,
    AVG(BLOCK_TIME_DIFF) AS AVG_TIME_DIFF,
  	PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY BLOCK_TIME_DIFF) AS MEDIAN_TIME_DIFF,
    MAX(BLOCK_TIME_DIFF) AS MAX_TIME_DIFF,
    MIN(BLOCK_TIME_DIFF) AS MIN_TIME_DIFF,
    AVG_TX_AMOUNT/AVG_TIME_DIFF AS AVG_SPEED,
  	AVG_TX_AMOUNT/MEDIAN_TIME_DIFF AS MEDIAN_SPEED
  FROM
  	BLOCK_TIME_TAB),

--------------------------------------------------ETHEREUM------------------------------------------------------------------------- 
ETHEREUM_PERFORMANCE AS (
  WITH BLOCKCHAIN_TX AS (
    SELECT
    	BLOCK_NUMBER,
    	COUNT(DISTINCT(TX_HASH)) AS TX_AMOUNT,
    	MIN(BLOCK_TIMESTAMP) AS BLOCK_TIME
    	--SUM(FEE_USD) AS TOTAL_FEE_USD
    FROM
     ethereum.core.fact_transactions
    WHERE DATE_TRUNC('day',BLOCK_TIMESTAMP) >= CURRENT_DATE - INTERVAL '30days'
    	--AND STATUS= 'SUCCESS' --EXPLAIN IT IN CONSTRAINTS AND FILTERS SECTION
    GROUP BY 1
    ORDER BY 2 ASC),
  
  BLOCK_TIME_TAB AS (
  SELECT
  	*,
    	LAG(BLOCK_TIME) OVER(ORDER BY BLOCK_TIME) AS PREVIOUS_BLOCK_TIME,
    	COALESCE(DATEDIFF('second',PREVIOUS_BLOCK_TIME,BLOCK_TIME),0) AS BLOCK_TIME_DIFF
  FROM BLOCKCHAIN_TX 
  ORDER BY 1)
  
  SELECT
    'L1' AS LAYER,
    'ETHEREUM' AS BLOCKCHAIN,
    AVG(TX_AMOUNT) AS AVG_TX_AMOUNT,
    MAX(TX_AMOUNT) AS MAX_TX_AMOUNT,
    MIN(TX_AMOUNT) AS MIN_TX_AMOUNT,
  	AVG(BLOCK_TIME_DIFF) AS AVG_TIME_DIFF,
    PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY BLOCK_TIME_DIFF) AS MEDIAN_TIME_DIFF,
    MAX(BLOCK_TIME_DIFF) AS MAX_TIME_DIFF,
    MIN(BLOCK_TIME_DIFF) AS MIN_TIME_DIFF,
    AVG_TX_AMOUNT/AVG_TIME_DIFF AS AVG_SPEED,
  	AVG_TX_AMOUNT/MEDIAN_TIME_DIFF AS MEDIAN_SPEED
  FROM
  	BLOCK_TIME_TAB),
  
 ------------------------------------------------OPTIMISM-----------------------------------------------------------
OPTIMISM_PERFORMANCE AS (
  WITH BLOCKCHAIN_TX AS (
    SELECT
    	BLOCK_NUMBER,
    	COUNT(DISTINCT(TX_HASH)) AS TX_AMOUNT,
    	MIN(BLOCK_TIMESTAMP) AS BLOCK_TIME
    	--SUM(FEE_USD) AS TOTAL_FEE_USD
    FROM
    optimism.core.fact_transactions
    WHERE DATE_TRUNC('day',BLOCK_TIMESTAMP) >= CURRENT_DATE - INTERVAL '30days'
    	--AND STATUS= 'SUCCESS' --EXPLAIN IT IN CONSTRAINTS AND FILTERS SECTION
    GROUP BY 1
    ORDER BY 2 ASC),
  
  BLOCK_TIME_TAB AS (
  SELECT
  	*,
    	LAG(BLOCK_TIME) OVER(ORDER BY BLOCK_TIME) AS PREVIOUS_BLOCK_TIME,
    	COALESCE(DATEDIFF('second',PREVIOUS_BLOCK_TIME,BLOCK_TIME),0) AS BLOCK_TIME_DIFF
  FROM BLOCKCHAIN_TX 
  ORDER BY 1)
  
  SELECT
    'L2' AS LAYER,
    'OPTIMISM' AS BLOCKCHAIN,
    AVG(TX_AMOUNT) AS AVG_TX_AMOUNT,
    MAX(TX_AMOUNT) AS MAX_TX_AMOUNT,
    MIN(TX_AMOUNT) AS MIN_TX_AMOUNT,
    AVG(BLOCK_TIME_DIFF) AS AVG_TIME_DIFF,
  	PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY BLOCK_TIME_DIFF) AS MEDIAN_TIME_DIFF,
    MAX(BLOCK_TIME_DIFF) AS MAX_TIME_DIFF,
    MIN(BLOCK_TIME_DIFF) AS MIN_TIME_DIFF,
    AVG_TX_AMOUNT/AVG_TIME_DIFF AS AVG_SPEED,
  	CASE
  		WHEN MEDIAN_TIME_DIFF = 0 THEN 1
  		ELSE  (AVG_TX_AMOUNT/MEDIAN_TIME_DIFF)
	END AS MEDIAN_SPEED
  FROM
  	BLOCK_TIME_TAB),

  ---------------------------------------------ARBITRUM------------------------------------------------------
ARBITRUM_PERFORMANCE AS (
  WITH BLOCKCHAIN_TX AS (
    SELECT
    	BLOCK_NUMBER,
    	COUNT(DISTINCT(TX_HASH)) AS TX_AMOUNT,
    	MIN(BLOCK_TIMESTAMP) AS BLOCK_TIME
    	--SUM(FEE_USD) AS TOTAL_FEE_USD
    FROM
   arbitrum.core.fact_transactions
    WHERE DATE_TRUNC('day',BLOCK_TIMESTAMP) >= CURRENT_DATE - INTERVAL '30days'
    	--AND STATUS= 'SUCCESS' --EXPLAIN IT IN CONSTRAINTS AND FILTERS SECTION
    GROUP BY 1
    ORDER BY 2 ASC),
  
  BLOCK_TIME_TAB AS (
  SELECT
  	*,
    	LAG(BLOCK_TIME) OVER(ORDER BY BLOCK_TIME) AS PREVIOUS_BLOCK_TIME,
    	COALESCE(DATEDIFF('second',PREVIOUS_BLOCK_TIME,BLOCK_TIME),0) AS BLOCK_TIME_DIFF
  FROM BLOCKCHAIN_TX 
  ORDER BY 1)
  
  SELECT
  'L2' AS LAYER,
  'ARBITRUM' AS BLOCKCHAIN,
  AVG(TX_AMOUNT) AS AVG_TX_AMOUNT,
  MAX(TX_AMOUNT) AS MAX_TX_AMOUNT,
  MIN(TX_AMOUNT) AS MIN_TX_AMOUNT,
  AVG(BLOCK_TIME_DIFF) AS AVG_TIME_DIFF,
  PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY BLOCK_TIME_DIFF) AS MEDIAN_TIME_DIFF,
  MAX(BLOCK_TIME_DIFF) AS MAX_TIME_DIFF,
  MIN(BLOCK_TIME_DIFF) AS MIN_TIME_DIFF,
  AVG_TX_AMOUNT/AVG_TIME_DIFF AS AVG_SPEED,
  CASE
  	WHEN MEDIAN_TIME_DIFF = 0 THEN 1
  	ELSE (AVG_TX_AMOUNT/MEDIAN_TIME_DIFF)
  END AS MEDIAN_SPEED
  FROM
  	BLOCK_TIME_TAB),

----------------------------------------------BSC-------------------------------------------------------------
BSC_PERFORMANCE AS (
  WITH BLOCKCHAIN_TX AS (
    SELECT
    	BLOCK_NUMBER,
    	COUNT(DISTINCT(TX_HASH)) AS TX_AMOUNT,
    	MIN(BLOCK_TIMESTAMP) AS BLOCK_TIME
    	--SUM(FEE_USD) AS TOTAL_FEE_USD
    FROM
    bsc.core.fact_transactions
    WHERE DATE_TRUNC('day',BLOCK_TIMESTAMP) >= CURRENT_DATE - INTERVAL '30days'
    	--AND STATUS= 'SUCCESS' --EXPLAIN IT IN CONSTRAINTS AND FILTERS SECTION
    GROUP BY 1
    ORDER BY 2 ASC),
  
  BLOCK_TIME_TAB AS (
  SELECT
  	*,
    	LAG(BLOCK_TIME) OVER(ORDER BY BLOCK_TIME) AS PREVIOUS_BLOCK_TIME,
    	COALESCE(DATEDIFF('second',PREVIOUS_BLOCK_TIME,BLOCK_TIME),0) AS BLOCK_TIME_DIFF
  FROM BLOCKCHAIN_TX 
  ORDER BY 1)
  
  SELECT
    'L2' AS LAYER,
    'BSC' AS BLOCKCHAIN,
    AVG(TX_AMOUNT) AS AVG_TX_AMOUNT,
    MAX(TX_AMOUNT) AS MAX_TX_AMOUNT,
    MIN(TX_AMOUNT) AS MIN_TX_AMOUNT,
  	AVG(BLOCK_TIME_DIFF) AS AVG_TIME_DIFF,
    PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY BLOCK_TIME_DIFF) AS MEDIAN_TIME_DIFF,
    MAX(BLOCK_TIME_DIFF) AS MAX_TIME_DIFF,
    MIN(BLOCK_TIME_DIFF) AS MIN_TIME_DIFF,
    AVG_TX_AMOUNT/AVG_TIME_DIFF AS AVG_SPEED,
  	AVG_TX_AMOUNT/MEDIAN_TIME_DIFF AS MEDIAN_SPEED
  FROM
  	BLOCK_TIME_TAB),

---------------------------------------------------------AVALANCHE--------------------------------------------------------
AVALANCHE_PERFORMANCE AS (
  WITH BLOCKCHAIN_TX AS (
    SELECT
    	BLOCK_NUMBER,
    	COUNT(DISTINCT(TX_HASH)) AS TX_AMOUNT,
    	MIN(BLOCK_TIMESTAMP) AS BLOCK_TIME
    	--SUM(FEE_USD) AS TOTAL_FEE_USD
    FROM
    avalanche.core.fact_transactions
    WHERE DATE_TRUNC('day',BLOCK_TIMESTAMP) >= CURRENT_DATE - INTERVAL '30days'
    	--AND STATUS= 'SUCCESS' --EXPLAIN IT IN CONSTRAINTS AND FILTERS SECTION
    GROUP BY 1
    ORDER BY 2 ASC),
  
  BLOCK_TIME_TAB AS (
  SELECT
  	*,
    	LAG(BLOCK_TIME) OVER(ORDER BY BLOCK_TIME) AS PREVIOUS_BLOCK_TIME,
    	COALESCE(DATEDIFF('second',PREVIOUS_BLOCK_TIME,BLOCK_TIME),0) AS BLOCK_TIME_DIFF
  FROM BLOCKCHAIN_TX 
  ORDER BY 1)
  
  SELECT
    'L2' AS LAYER,
    'AVALANCHE' AS BLOCKCHAIN,
    AVG(TX_AMOUNT) AS AVG_TX_AMOUNT,
    MAX(TX_AMOUNT) AS MAX_TX_AMOUNT,
    MIN(TX_AMOUNT) AS MIN_TX_AMOUNT,
   	AVG(BLOCK_TIME_DIFF) AS AVG_TIME_DIFF,
    PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY BLOCK_TIME_DIFF) AS MEDIAN_TIME_DIFF,
    MAX(BLOCK_TIME_DIFF) AS MAX_TIME_DIFF,
    MIN(BLOCK_TIME_DIFF) AS MIN_TIME_DIFF,
   AVG_TX_AMOUNT/AVG_TIME_DIFF AS AVG_SPEED,
  	AVG_TX_AMOUNT/MEDIAN_TIME_DIFF AS MEDIAN_SPEED
  FROM
  	BLOCK_TIME_TAB),

-----------------------------------------------OSMOSIS----------------------------------------------------
OSMOSIS_PERFORMANCE AS (
  WITH BLOCKCHAIN_TX AS (
    SELECT
    	BLOCK_ID,
    	COUNT(DISTINCT(TX_ID)) AS TX_AMOUNT,
    	MIN(BLOCK_TIMESTAMP) AS BLOCK_TIME
    	--SUM(FEE_USD) AS TOTAL_FEE_USD
    FROM
     osmosis.core.fact_transactions
    WHERE DATE_TRUNC('day',BLOCK_TIMESTAMP) >= CURRENT_DATE - INTERVAL '30days'
    	--AND TX_STATUS= 'SUCCEEDED' --EXPLAIN IT IN CONSTRAINTS AND FILTERS SECTION
    GROUP BY 1
    ORDER BY 2 ASC),
  
  BLOCK_TIME_TAB AS (
  SELECT
  	*,
    	LAG(BLOCK_TIME) OVER(ORDER BY BLOCK_TIME) AS PREVIOUS_BLOCK_TIME,
    	COALESCE(DATEDIFF('second',PREVIOUS_BLOCK_TIME,BLOCK_TIME),0) AS BLOCK_TIME_DIFF
  FROM BLOCKCHAIN_TX 
  ORDER BY 1)
  
  SELECT
    'L1' AS LAYER,
    'OSMOSIS' AS BLOCKCHAIN,
    AVG(TX_AMOUNT) AS AVG_TX_AMOUNT,
    MAX(TX_AMOUNT) AS MAX_TX_AMOUNT,
    MIN(TX_AMOUNT) AS MIN_TX_AMOUNT,
  	AVG(BLOCK_TIME_DIFF) AS AVG_TIME_DIFF,
    PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY BLOCK_TIME_DIFF) AS MEDIAN_TIME_DIFF,
    MAX(BLOCK_TIME_DIFF) AS MAX_TIME_DIFF,
    MIN(BLOCK_TIME_DIFF) AS MIN_TIME_DIFF,
    AVG_TX_AMOUNT/AVG_TIME_DIFF AS AVG_SPEED,
  	AVG_TX_AMOUNT/MEDIAN_TIME_DIFF AS MEDIAN_SPEED
  FROM
  	BLOCK_TIME_TAB),

------------------------------------- ALGORAND --------------------------------------------------------------------
 ALGORAND_PERFORMANCE AS (
  WITH BLOCKCHAIN_TX AS (
    SELECT
    	BLOCK_ID,
    	COUNT(DISTINCT(TX_ID)) AS TX_AMOUNT,
    	MIN(BLOCK_TIMESTAMP) AS BLOCK_TIME
    	--SUM(FEE_USD) AS TOTAL_FEE_USD
    FROM
    flipside_prod_db.algorand.transactions
    WHERE DATE_TRUNC('day',BLOCK_TIMESTAMP) >= CURRENT_DATE - INTERVAL '30days'
    	--AND TX_STATUS= 'SUCCEEDED' --EXPLAIN IT IN CONSTRAINTS AND FILTERS SECTION
    GROUP BY 1
    ORDER BY 2 ASC),
  
  BLOCK_TIME_TAB AS (
  SELECT
  	*,
    	LAG(BLOCK_TIME) OVER(ORDER BY BLOCK_TIME) AS PREVIOUS_BLOCK_TIME,
    	COALESCE(DATEDIFF('second',PREVIOUS_BLOCK_TIME,BLOCK_TIME),0) AS BLOCK_TIME_DIFF
  FROM BLOCKCHAIN_TX 
  ORDER BY 1)
  
  SELECT
    'L1' AS LAYER,
    'ALGORAND' AS BLOCKCHAIN,
    AVG(TX_AMOUNT) AS AVG_TX_AMOUNT,
    MAX(TX_AMOUNT) AS MAX_TX_AMOUNT,
    MIN(TX_AMOUNT) AS MIN_TX_AMOUNT,
  	AVG(BLOCK_TIME_DIFF) AS AVG_TIME_DIFF,
    PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY BLOCK_TIME_DIFF) AS MEDIAN_TIME_DIFF,
    MAX(BLOCK_TIME_DIFF) AS MAX_TIME_DIFF,
    MIN(BLOCK_TIME_DIFF) AS MIN_TIME_DIFF,
    AVG_TX_AMOUNT/AVG_TIME_DIFF AS AVG_SPEED,
  	AVG_TX_AMOUNT/MEDIAN_TIME_DIFF AS MEDIAN_SPEED
  FROM
  	BLOCK_TIME_TAB),
 
------------------------------------------- CONCATENATE ALL TABLES ---------------------------------------------- 
ALL_TABLES AS (
  (SELECT * FROM POLYGON_PERFORMANCE)
  UNION
  (SELECT * FROM ETHEREUM_PERFORMANCE)
  UNION
  (SELECT * FROM SOLANA_PERFORMANCE)
  UNION
  (SELECT * FROM FLOW_PERFORMANCE)
  UNION
  (SELECT * FROM ALGORAND_PERFORMANCE)
  UNION
  (SELECT * FROM OSMOSIS_PERFORMANCE)
  UNION
  (SELECT * FROM BSC_PERFORMANCE)
  UNION
  (SELECT * FROM AVALANCHE_PERFORMANCE)
  UNION
  (SELECT * FROM OPTIMISM_PERFORMANCE)
  UNION
  (SELECT * FROM ARBITRUM_PERFORMANCE))

SELECT *
FROM ALL_TABLES
ORDER BY AVG_TIME_DIFF DESC
"""

In [6]:
df_blocks = ShroomDK(api_key,query)

In [7]:
df_blocks

,LAYER,BLOCKCHAIN,AVG_TX_AMOUNT,MAX_TX_AMOUNT,MIN_TX_AMOUNT,AVG_TIME_DIFF,MEDIAN_TIME_DIFF,MAX_TIME_DIFF,MIN_TIME_DIFF,AVG_SPEED,MEDIAN_SPEED
0,L2,BSC,105.158685,633,15,337.420070,3,57190,0,0.311655,35.052895
1,L1,ETHEREUM,192.798176,1351,1,14.075827,10,146,0,13.697112,19.279818
2,L1,OSMOSIS,43.092042,521,1,6.520540,6,679,0,6.608662,7.182007
3,L1,ALGORAND,48.949198,4966,1,4.335299,4,66,0,11.290847,12.237300
4,L2,POLYGON,73.870618,1160,1,2.741934,2,3511,0,26.941063,36.935309
5,L2,AVALANCHE,4.980542,360,1,2.021291,2,135,0,2.464040,2.490271
6,L1,FLOW,13.451814,435,1,1.531609,1,52774,0,8.782799,13.451814
7,L2,ARBITRUM,1.121771,125,1,0.998764,0,219,0,1.123159,1.000000
8,L2,OPTIMISM,1.000000,1,1,0.935677,0,46,0,1.068745,1.000000
9,L1,SOLANA,400.686695,1523,1,0.717122,1,84,0,558.742717,400.686695


### 2. Transactions per Time (Speed) Overtime

In [ ]:
networks = {
    'Polygon': {},
    'Solana': {},
    'FLOW': {},
    'Ethereum': {},
    'Optimism': {},
    'Arbitrum': {},
    'BSC': {},
    'Avalanche': {},
    'Osmosis': {},
    'Algorand': {}
}

query = """
WITH BLOCKCHAIN_TX AS (
    SELECT
    	BLOCK_ID,
    	COUNT(DISTINCT(TX_ID)) AS TX_AMOUNT,
    	MIN(BLOCK_TIMESTAMP) AS BLOCK_TIME
    	--SUM(FEE_USD) AS TOTAL_FEE_USD
    FROM
     flipside_prod_db.polygon.transactions
    WHERE DATE_TRUNC('day',BLOCK_TIMESTAMP) >= CURRENT_DATE - INTERVAL '30days'
    	--AND SUCCESS = TRUE --EXPLAIN IT IN CONSTRAINTS AND FILTERS SECTION
    GROUP BY 1
    ORDER BY 2 ASC),
  
  BLOCK_TIME_TAB AS (
  SELECT
  	*,
    	LAG(BLOCK_TIME) OVER(ORDER BY BLOCK_TIME) AS PREVIOUS_BLOCK_TIME,
    	COALESCE(DATEDIFF('second',PREVIOUS_BLOCK_TIME,BLOCK_TIME),0) AS BLOCK_TIME_DIFF,
  		CASE
  		 	WHEN BLOCK_TIME_DIFF = 0 THEN 0
  			ELSE (TX_AMOUNT/BLOCK_TIME_DIFF)
  		END AS SPEED
  FROM BLOCKCHAIN_TX 
  ORDER BY 1)
  
  SELECT
  DATE_TRUNC('day',	block_time) as DATE,
  AVG(SPEED) AS AVG_DAILY_SPEED
  FROM
  	BLOCK_TIME_TAB
  GROUP BY 1
ORDER BY 1
"""